In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LDA Modeling

In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
import time
import pdb

from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from gensim.models import LdaModel
import pyLDAvis.gensim               #pyLDAvis.gensim_models
from scipy.spatial import distance

#nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
test = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
title = pd.read_csv('/kaggle/input/cpc-codes/titles.csv')
train_len = len(train)
test_len = len(test)
total_len = train_len + test_len
print(total_len, train_len, test_len)

In [ ]:
frames = [train, test]
all = pd.concat(frames, keys=['train', 'test'])
all

In [ ]:
df_test = all.loc['test']
df_test

In [ ]:
df_LEFT_JOIN = pd.merge(all, title, left_on='context', right_on='code', how='left')
df_LEFT_JOIN

In [ ]:
data = df_LEFT_JOIN.drop(['code','section','class','subclass','group','main_group'], axis=1)
data = data[['id','anchor','target','context','title','score']]
data

In [ ]:
def clean_text(d):
    d = str(d)
    pattern = r'[^a-zA-Z\'\s]'
    text = re.sub(pattern, '', d)
    text = text.lower()
    #text = [word for word in text if len(word)>2]
    return ' '.join([w for w in text.split() if len(w) > 3])# 3이하인거 버리기

def preprocessing(d):
    return preprocess_string(d)

def data_load(data):
    df = pd.DataFrame({'text': data['total']})
    tokenized_text = df['text'].apply(preprocessing)
    tokenized_text = tokenized_text.to_list()
    return tokenized_text 

def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:  
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
            else:
                break
    return(topic_table)

def create_output(topictable, topicnum):
    total_topic_list=list()
    
    for topic in range(len(topictable["각 토픽의 비중"])):
        topic_dict=dict()
        tmp_list = list()
        for k in range(len(topictable["각 토픽의 비중"][topic])):
            topic_dict[f'topic_{int(topictable["각 토픽의 비중"][topic][k][0])}'] = float(topictable["각 토픽의 비중"][topic][k][1])
            tmp_list.append(int(topictable["각 토픽의 비중"][topic][k][0]))

        num_list = [i for i in range(topicnum)]
        num_list = [i for i in num_list if i not in tmp_list]   ## 있는 토픽 제거

        for n in num_list:
            topic_dict[f'topic_{n}'] = 0.0      ## 확률 0인 토픽 넣어줌
        sorted_dict = sorted(topic_dict.items())
        total_topic_list.append(dict(sorted_dict))
        
    return total_topic_list


In [ ]:
data['title'] = data['title'].apply(clean_text)

In [ ]:
origin_data = data
origin_data

In [ ]:
total_df = pd.DataFrame(columns=['id','total'])
total_df['total'] = data['anchor'] + " " + data['target'] + " " +  data['title']
total_df['id'] = data['id']
total_df

In [ ]:
doc_token = data_load(total_df)
LDA_new_list = doc_token

## Modeling
dictionary = corpora.Dictionary(doc_token)
corpus = [dictionary.doc2bow(text) for text in doc_token] #문서를 bag-of-words 형태로 바꾼것

NUM_TOPICS = 9
NUM_PASSES = 30
lda_model =LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes = NUM_PASSES)
topics = lda_model.print_topics(num_words = 10)
    
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'LDAvis_t9_total1.html')

topictable = make_topictable_per_doc(lda_model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

topic_list = create_output(topictable, NUM_TOPICS)

In [ ]:
topic_list[0]

# Using BERT

In [ ]:
train_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
cpc = pd.read_csv('../input/cpc-codes/titles.csv')
cpc = cpc.rename(columns = {"code" : "context"})
train_df = pd.merge(train_df, cpc[["context","title"]], on ="context", how = "left")

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string

from tqdm import tqdm

import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader, Dataset, TensorDataset, SequentialSampler, RandomSampler

def normalize_text(s):

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    
    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))

train_df['title'] = train_df['title'].apply(lambda x: normalize_text(x))
train_df['sen1'] = train_df['anchor'].astype('str')+' '+train_df['title'].astype('str')
train_df = train_df.drop(['anchor','context','title'],axis=1)

In [ ]:
lda_res = []
for res in topic_list:
    temp = []
    for val in res.values():
        temp.append(val)
    lda_res.append(str(temp)[1:-1])

train_lda = lda_res[:train_len]
test_lda = lda_res[-test_len:]
print(len(lda_res), len(train_lda), len(test_lda))
print(train_lda[0])

In [ ]:
train_df['lda_dist'] = train_lda

In [ ]:
train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(train_df[['target','sen1', 'lda_dist']],train_df['score'],random_state=1234,test_size=0.1)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

In [ ]:
import torch
import torch.nn as nn
import transformers
from torch.nn.utils.clip_grad import clip_grad_norm


torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
class BERT(nn.Module):
    def __init__(self,bert_path):
        super(BERT,self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.AutoModel.from_pretrained(self.bert_path)
        self.fc_layer = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(1033,1),
            nn.Sigmoid()
        )
    def forward(self,ids,mask,token_type_ids, batch_lda):
        out = self.bert(input_ids=ids,attention_mask=mask,token_type_ids=token_type_ids,return_dict=True)
        pooler_output = out.get('pooler_output')
        pooler_output = torch.cat([pooler_output, batch_lda], dim=1)
        cls_out = self.fc_layer(pooler_output)
        return cls_out

In [ ]:
class Train_dataset:
    def __init__(self,text1,text2,label, tokenizer,max_len, lda_res):
        self.text1=text1
        self.text2=text2
        self.label=label
        self.lda_res = lda_res
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text1)
    
    def __getitem__(self,idx):
        text_1 = str(self.text1[idx])
        text_2 = str(self.text2[idx])
        label = self.label[idx]
        lda = self.lda_res[idx]
        
        inputs = self.tokenizer(
            text_1,
            text_2,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_attention_mask=True
        )
        
        ids = inputs['input_ids']
        token_type_ids = inputs["token_type_ids"]
        mask = inputs['attention_mask']
        
        padding_len = self.max_len - len(ids)
        ids = ids + ([0]*padding_len)
        token_type_ids = token_type_ids + ([0]*padding_len)
        mask = mask + ([0]*padding_len)
        
        return {
            "ids": torch.tensor(ids,dtype=torch.long),
            "mask": torch.tensor(mask,dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids,dtype=torch.long),
            "targets": torch.tensor(label,dtype=torch.float),
            'lda': torch.tensor(lda, dtype=torch.float)
        }

In [ ]:
train_lda[0]

In [ ]:
max_len=128
train_batch_size = 16
epochs=5
bert_path = '../input/bert-for-patents/bert-for-patents'

tokenizer = transformers.AutoTokenizer.from_pretrained(bert_path)

# Training dataset prep

train_text1 = list(x_train['target'].values)
train_text2 = list(x_train['sen1'].values)
train_label = list(y_train.values)
train_lda = list(x_train['lda_dist'].values)
train_lda = [ list(map(float, lda.split(', ')))  for lda in train_lda]


train_dataset = Train_dataset(
    text1 = train_text1,
    text2 = train_text2,
    label = train_label,
    lda_res = train_lda,
    tokenizer=tokenizer ,
    max_len=max_len
)

train_data_loader = DataLoader(train_dataset,batch_size=train_batch_size,shuffle=True)

# validation dataset prep
val_text1 = list(x_test['target'].values)
val_text2 = list(x_test['sen1'].values)
val_label = list(y_test.values)
val_lda = list(x_test['lda_dist'].values)
val_lda = [ list(map(float, lda.split(', ')))  for lda in val_lda]

valid_dataset = Train_dataset(
    text1 = val_text1,
    text2 = val_text2,
    label = val_label,
    lda_res = val_lda,
    tokenizer=tokenizer,
    max_len=max_len
)

valid_data_loader = DataLoader(valid_dataset,batch_size=train_batch_size,shuffle=True)

In [ ]:
def train(model, optimizer, scheduler, loss_function, epochs,train_dataloader, device, clip_value=2):
    model.train()
    for epoch in range(epochs):
        best_loss = []
        for step, batch in enumerate(train_dataloader): 
            batch_inputs, batch_masks, batch_labels = batch['ids'].to(device), batch['mask'].to(device), batch['targets'].to(device)
            batch_lda = batch['lda'].to(device)
            batch_token_type_ids = batch['token_type_ids'].to(device)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks, batch_token_type_ids, batch_lda)
            loss = loss_function(outputs.squeeze(),batch_labels.squeeze())
            best_loss.append(loss)
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()

        loss2 = sum(best_loss)/len(best_loss)
        print(f'Epoch : {epoch} ,Train loss : {loss2}')
                
    return model

def compute_metrics(predictions, labels):

    predictions = predictions.reshape(len(predictions))
    predictions = predictions.cpu().clone().numpy()
    labels = labels.cpu().clone().numpy()
    return  np.corrcoef(predictions, labels)[0][1]

def evaluate(model,loss_function,test_dataloader,device):
    model.eval()
    test_loss, test_pear = [], []
    for step,batch in enumerate(test_dataloader):
        batch_inputs, batch_masks, batch_labels = batch['ids'].to(device), batch['mask'].to(device), batch['targets'].to(device)
        batch_token_type_ids = batch['token_type_ids'].to(device)
        batch_lda = batch['lda'].to(device)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks, batch_token_type_ids, batch_lda)
        loss = loss_function(outputs, batch_labels)
        test_loss.append(loss.item())
        pearson = compute_metrics(outputs, batch_labels)
        test_pear.append(pearson.item())
    return test_loss, test_pear

In [ ]:
num_train_steps = len(train_data_loader) * epochs

model = BERT(bert_path).to(device)

optimizer = transformers.AdamW(model.parameters(),lr=3e-5,eps=1e-8)

scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

loss_function = nn.MSELoss()


model = train(model, optimizer, scheduler, loss_function, epochs,train_data_loader, device)


loss1,pear_ = evaluate(model,loss_function,valid_data_loader,device)

loss = sum(loss1)/len(loss1)
pear = sum(pear_)/len(pear_)
print(f"eval mean result : loss {loss}, pearson {pear}")

In [ ]:
torch.save(model.state_dict(),f'./my_bert')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
test_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
cpc = pd.read_csv('../input/cpc-codes/titles.csv')
cpc = cpc.rename(columns = {"code" : "context"})
test_df = pd.merge(test_df, cpc[["context","title"]], on ="context", how = "left")
test_df['title'] = test_df['title'].apply(lambda x: normalize_text(x))
test_df['sen1'] = test_df['anchor'].astype('str')+' '+test_df['title'].astype('str')
test_df = test_df.drop(['anchor','context','title'],axis=1)
test_df['lda_dist'] = test_lda

In [ ]:
test_df.head()

In [ ]:
class Test_datset:
    def __init__(self,text1,text2,idf, tokenizer,max_len, lda_res):
        self.text1=text1
        self.text2=text2
        self.idf = idf
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.lda_res = lda_res
        
    def __len__(self):
        return len(self.text1)
    
    def __getitem__(self,idx):
        text_1 = str(self.text1[idx])
        text_2 = str(self.text2[idx])
        idf = self.idf[idx]
        lda = self.lda_res[idx]
        
        inputs = self.tokenizer(
            text_1,
            text_2,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_attention_mask=True
        )
        
        ids = inputs['input_ids']
        token_type_ids = inputs["token_type_ids"]
        mask = inputs['attention_mask']
        
        padding_len = self.max_len - len(ids)
        ids = ids + ([0]*padding_len)
        token_type_ids = token_type_ids + ([0]*padding_len)
        mask = mask + ([0]*padding_len)
        
        return {
            "ids": torch.tensor(ids,dtype=torch.long),
            "mask": torch.tensor(mask,dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids,dtype=torch.long),
            "idf": idf,
            'lda': torch.tensor(lda, dtype=torch.float)
        }

In [ ]:
test_text1 = list(test_df['target'].values)
test_text2 = list(test_df['sen1'].values)
test_lda = list(test_df['lda_dist'].values) 
test_lda = [ list(map(float, lda.split(', ')))  for lda in test_lda]

test_dataset = Test_datset(
    text1 = test_text1,
    text2 = test_text2,
    idf=list(test_df['id'].values),
    tokenizer=tokenizer,
    max_len=max_len,
    lda_res = test_lda
)

test_data_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)

In [ ]:
def predict(model,test_dataloader,device):
    model.eval()
    result = []
    for step,batch in enumerate(test_dataloader):
        batch_inputs, batch_masks = batch['ids'].to(device), batch['mask'].to(device)
        batch_lda = batch['lda'].to(device)
        batch_token_type_ids = batch['token_type_ids'].to(device)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks, batch_token_type_ids, batch_lda )
        out = [i[0] for i in outputs.cpu().detach().numpy()]
        batch_idf = batch['idf']
        temp = [[i,j] for i,j in zip(batch_idf,out)]
        result.extend(temp)
    return result

In [ ]:
model = BERT(bert_path).to(device)
model.load_state_dict(torch.load('my_bert'))

final_res = predict(model,test_data_loader,device)
submit_csv = pd.DataFrame(final_res,columns=['id','score'])
submit_csv.to_csv('submission.csv',index=False)